# Importing libraries

In [29]:
import pandas as pd     # for calculation
import numpy as np

from sklearn.experimental import enable_iterative_imputer   # for mssing values calculation
from sklearn.impute import IterativeImputer
from sklearn.ensemble import IsolationForest

import matplotlib.pyplot as plt     # for graph

from sklearn.cluster import DBSCAN  # for outliers

# User-Defined Function (UDF)

In [30]:
# creating a function for monthly calculation from yearly and weakly 
def monthly(x, value):
    if x in [409,419, 619, 649]:
        return (value * 30)/ 365
    elif x in [169, 219, 239, 249, 189, 269, 279, 199, 289, 299, 309, 319, 329]:
        return (value * 30)/ 7
    else:
        return value

In [31]:
# creating a function for sector and state spliting from Common-id
def split_string(s):
    part0 = s[:13]   # first 13 bytes
    part1 = s[13:14]  # next 2 bytes
    part2 = s[14:16]  # next 2 bytes
    part3 = s[16:]   # remaining bytes
    return pd.Series([part0,part1, part2, part3]) 

# Importing data

In [32]:
# Importing level 2 data from divice
col_specification=[(0,38),(38,39),(39,41),(41,43),(43,44),(44,45),(45,48),(48,49),(49,51),(51,53),(53,54),(54,56),(56,57),(57,59),(59,61),(61,63),(63,65),(65,67),(67,68),(68,69),(69,84)]
col_names=['Common-ID','Questionnaire No.','Level','Person Srl No.','Relation to head (code)','Gender','Age(in years)','Marital Status (code)','Highest educational level attained (code)','Total year of education completed','Whether used internet from any location during last 30 days','No. of days stayed away from home during last 30 days','No. of meals usually taken in a day','No. of meals taken during last 30 days from school, balwadi etc.','No. of meals taken during last 30 days  others','No. of meals taken during last 30 days from employer as perquisites or part of wage','No. of meals taken during last 30 days at home','No. of meals taken during last 30 days on payment','FDQ original member(generated field)','Status of Member as on revisit','Multiplier']
level_2 = pd.read_fwf(r'F:\DATA\D_work\Interpreation\hces22_lvl_02.txt', header=None, colspecs=col_specification, names = col_names)

# As provided in "Read me" text dividing multiplier by 100
level_2['Multiplier'] = level_2['Multiplier']/100
level_2.head(5)

,Common-ID,Questionnaire No.,Level,Person Srl No.,Relation to head (code),Gender,Age(in years),Marital Status (code),Highest educational level attained (code),Total year of education completed,...,No. of days stayed away from home during last 30 days,No. of meals usually taken in a day,"No. of meals taken during last 30 days from school, balwadi etc.",No. of meals taken during last 30 days others,No. of meals taken during last 30 days from employer as perquisites or part of wage,No. of meals taken during last 30 days at home,No. of meals taken during last 30 days on payment,FDQ original member(generated field),Status of Member as on revisit,Multiplier
0,HCES2022655561010131113011 101202 201,H,2,1,1,1,48,2,6,12.0,...,0.0,2.0,0.0,0.0,0.0,0.0,58.0,1,1.0,355.6
1,HCES2022655561010131113011 101202 201,H,2,2,2,2,46,2,1,NaN,...,0.0,2.0,NaN,NaN,NaN,NaN,60.0,1,1.0,355.6
2,HCES2022655561010131113011 101202 201,H,2,3,5,1,24,1,13,18.0,...,0.0,2.0,NaN,NaN,NaN,NaN,58.0,1,1.0,355.6
3,HCES2022655561010131113011 101202 201,H,2,4,5,1,18,1,7,13.0,...,0.0,2.0,NaN,NaN,NaN,NaN,56.0,1,1.0,355.6
4,HCES2022655561010131113011 101202 201,H,2,5,5,2,21,1,12,17.0,...,0.0,2.0,NaN,NaN,NaN,NaN,54.0,1,1.0,355.6


In [33]:
z =level_2.groupby('Common-ID')['Multiplier'].unique().reset_index()

In [34]:
z.head()

,Common-ID,Multiplier
0,HCES2022310002282831212025 228111 301,[964.98]
1,HCES2022310002282831212025 228111 302,[964.98]
2,HCES2022310002282831212025 228111 303,[964.98]
3,HCES2022310002282831212025 228111 304,[964.98]
4,HCES2022310002282831212025 228111 305,[964.98]


In [35]:
z[['part2','Sector', 'State', 'part3']] = z['Common-ID'].apply(split_string)

In [36]:
z.head()

,Common-ID,Multiplier,part2,Sector,State,part3
0,HCES2022310002282831212025 228111 301,[964.98],HCES202231000,2,28,2831212025 228111 301
1,HCES2022310002282831212025 228111 302,[964.98],HCES202231000,2,28,2831212025 228111 302
2,HCES2022310002282831212025 228111 303,[964.98],HCES202231000,2,28,2831212025 228111 303
3,HCES2022310002282831212025 228111 304,[964.98],HCES202231000,2,28,2831212025 228111 304
4,HCES2022310002282831212025 228111 305,[964.98],HCES202231000,2,28,2831212025 228111 305


In [37]:
# Importing level-14 data from device
col_specification=[(0,38),(38,39),(39,41),(41,46),(46,49),(49,59),(59,74)]
col_names=['Common-ID','Questionnaire No.','Level','Section','Item Code','Value (in Rs)','Multiplier']
lvl_14= pd.read_fwf(r'F:\DATA\D_work\Interpreation\hces22_lvl_14.txt', header=None, colspecs=col_specification, names = col_names)

# As provided in "Read me" text dividing multiplier by 100
lvl_14['Multiplier'] = lvl_14['Multiplier']/100
lvl_14.head(5)

,Common-ID,Questionnaire No.,Level,Section,Item Code,Value (in Rs),Multiplier
0,HCES2022655621010121713017 101111 301,F,14,5.1,139,90,88.9
1,HCES2022655621010121713017 101111 310,F,14,5.1,139,90,88.9
2,HCES2022655621010121713017 101111 311,F,14,5.1,139,25,88.9
3,HCES2022655621010121713017 101111 313,F,14,5.1,139,45,88.9
4,HCES2022655371010122023014 201111 202,F,14,5.1,139,75,1405.0


### Primary cleanning
Splitting the non-usable column and multiplier divided by 100 as defined in the "read me" word file

In [38]:
# for counting total number of members of a household
lvl_2 = level_2.value_counts('Common-ID').to_frame().reset_index()
lvl_2.head(5)

,Common-ID,count
0,HCES2022666371080811523175 108112 107,37
1,HCES2022341462363612342036 236101 1312,31
2,HCES2022636211090934223383 109205 303,30
3,HCES2022670621353510123017 235101 203,29
4,HCES2022636811090920723403 109125 308,28


In [39]:
mu = pd.merge(lvl_2,z, on='Common-ID',how='left')
mu.head()

,Common-ID,count,Multiplier,part2,Sector,State,part3
0,HCES2022666371080811523175 108112 107,37,[1852.97],HCES202266637,1,08,0811523175 108112 107
1,HCES2022341462363612342036 236101 1312,31,[1859.42],HCES202234146,2,36,3612342036 236101 1312
2,HCES2022636211090934223383 109205 303,30,[2087.25],HCES202263621,1,09,0934223383 109205 303
3,HCES2022670621353510123017 235101 203,29,[88.19],HCES202267062,1,35,3510123017 235101 203
4,HCES2022636811090920723403 109125 308,28,[1394.85],HCES202263681,1,09,0920723403 109125 308


In [40]:
len(mu)

261746

In [41]:
mu['total_mambers'] = mu['count']*mu['Multiplier']
mu.head(5)

,Common-ID,count,Multiplier,part2,Sector,State,part3,total_mambers
0,HCES2022666371080811523175 108112 107,37,[1852.97],HCES202266637,1,08,0811523175 108112 107,[68559.89]
1,HCES2022341462363612342036 236101 1312,31,[1859.42],HCES202234146,2,36,3612342036 236101 1312,[57642.020000000004]
2,HCES2022636211090934223383 109205 303,30,[2087.25],HCES202263621,1,09,0934223383 109205 303,[62617.5]
3,HCES2022670621353510123017 235101 203,29,[88.19],HCES202267062,1,35,3510123017 235101 203,[2557.5099999999998]
4,HCES2022636811090920723403 109125 308,28,[1394.85],HCES202263681,1,09,0920723403 109125 308,[39055.799999999996]


In [42]:
# Dropping level and Section from data frame (lvl_14)
level_14 = lvl_14.drop(['Level','Section'],axis=1)
level_14.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier
0,HCES2022655621010121713017 101111 301,F,139,90,88.9
1,HCES2022655621010121713017 101111 310,F,139,90,88.9
2,HCES2022655621010121713017 101111 311,F,139,25,88.9
3,HCES2022655621010121713017 101111 313,F,139,45,88.9
4,HCES2022655371010122023014 201111 202,F,139,75,1405.0


In [43]:
main_data = pd.merge(level_14,mu, on='Common-ID', how='left')
main_data.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier_x,count,Multiplier_y,part2,Sector,State,part3,total_mambers
0,HCES2022655621010121713017 101111 301,F,139,90,88.9,5,[88.9],HCES202265562,1,01,0121713017 101111 301,[444.5]
1,HCES2022655621010121713017 101111 310,F,139,90,88.9,5,[88.9],HCES202265562,1,01,0121713017 101111 310,[444.5]
2,HCES2022655621010121713017 101111 311,F,139,25,88.9,6,[88.9],HCES202265562,1,01,0121713017 101111 311,[533.4000000000001]
3,HCES2022655621010121713017 101111 313,F,139,45,88.9,5,[88.9],HCES202265562,1,01,0121713017 101111 313,[444.5]
4,HCES2022655371010122023014 201111 202,F,139,75,1405.0,4,[1405.0],HCES202265537,1,01,0122023014 201111 202,[5620.0]


In [44]:
len(main_data)

8075528

In [45]:
main_data['total_mambers'] = main_data['total_mambers'].astype(int)
main_data.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier_x,count,Multiplier_y,part2,Sector,State,part3,total_mambers
0,HCES2022655621010121713017 101111 301,F,139,90,88.9,5,[88.9],HCES202265562,1,01,0121713017 101111 301,444
1,HCES2022655621010121713017 101111 310,F,139,90,88.9,5,[88.9],HCES202265562,1,01,0121713017 101111 310,444
2,HCES2022655621010121713017 101111 311,F,139,25,88.9,6,[88.9],HCES202265562,1,01,0121713017 101111 311,533
3,HCES2022655621010121713017 101111 313,F,139,45,88.9,5,[88.9],HCES202265562,1,01,0121713017 101111 313,444
4,HCES2022655371010122023014 201111 202,F,139,75,1405.0,4,[1405.0],HCES202265537,1,01,0122023014 201111 202,5620


In [46]:
level_14.count()

Common-ID            8075528
Questionnaire No.    8075528
Item Code            8075528
Value (in Rs)        8075528
Multiplier           8075528
dtype: int64

## Appling monthly function
so we can read monthly expendeture

In [47]:
# Creating a new column for monthly calculated values using monthly function
main_data['Monthly Value'] = main_data.apply(lambda row: monthly(row['Item Code'], row['Value (in Rs)']), axis=1)
main_data.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier_x,count,Multiplier_y,part2,Sector,State,part3,total_mambers,Monthly Value
0,HCES2022655621010121713017 101111 301,F,139,90,88.9,5,[88.9],HCES202265562,1,01,0121713017 101111 301,444,90.0
1,HCES2022655621010121713017 101111 310,F,139,90,88.9,5,[88.9],HCES202265562,1,01,0121713017 101111 310,444,90.0
2,HCES2022655621010121713017 101111 311,F,139,25,88.9,6,[88.9],HCES202265562,1,01,0121713017 101111 311,533,25.0
3,HCES2022655621010121713017 101111 313,F,139,45,88.9,5,[88.9],HCES202265562,1,01,0121713017 101111 313,444,45.0
4,HCES2022655371010122023014 201111 202,F,139,75,1405.0,4,[1405.0],HCES202265537,1,01,0122023014 201111 202,5620,75.0


# One item study
For checking data is representing correctly or not we only study 'Item code' 169 for milk and milk products 

In [48]:
# Filtering the data-frame for checking milk & milk prodect consumption in a single housee hold
##checking if monthly funtion is functioning properly
todoroki = main_data[main_data['Item Code'].isin([239])]
temp1 = todoroki.copy()
temp1.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier_x,count,Multiplier_y,part2,Sector,State,part3,total_mambers,Monthly Value
12877,HCES2022653651030311613011 103123 2301,F,239,50,1501.65,4,[1501.65],HCES202265365,1,03,0311613011 103123 2301,6006,214.285714
12887,HCES2022653651030311613011 103123 2302,F,239,60,1501.65,3,[1501.65],HCES202265365,1,03,0311613011 103123 2302,4504,257.142857
12914,HCES2022653651030311613011 103123 2306,F,239,60,1501.65,2,[1501.65],HCES202265365,1,03,0311613011 103123 2306,3003,257.142857
12989,HCES2022653651030311613011 103123 2316,F,239,50,1501.65,5,[1501.65],HCES202265365,1,03,0311613011 103123 2316,7508,214.285714
13007,HCES2022653651030311613011 103123 2318,F,239,110,1501.65,6,[1501.65],HCES202265365,1,03,0311613011 103123 2318,9009,471.428571


In [49]:
# description of monthly value only for milk & milk prodects ('239') 
temp1['total_Value'] = temp1['Monthly Value']*temp1['Multiplier_x']
temp1.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier_x,count,Multiplier_y,part2,Sector,State,part3,total_mambers,Monthly Value,total_Value
12877,HCES2022653651030311613011 103123 2301,F,239,50,1501.65,4,[1501.65],HCES202265365,1,03,0311613011 103123 2301,6006,214.285714,321782.142857
12887,HCES2022653651030311613011 103123 2302,F,239,60,1501.65,3,[1501.65],HCES202265365,1,03,0311613011 103123 2302,4504,257.142857,386138.571429
12914,HCES2022653651030311613011 103123 2306,F,239,60,1501.65,2,[1501.65],HCES202265365,1,03,0311613011 103123 2306,3003,257.142857,386138.571429
12989,HCES2022653651030311613011 103123 2316,F,239,50,1501.65,5,[1501.65],HCES202265365,1,03,0311613011 103123 2316,7508,214.285714,321782.142857
13007,HCES2022653651030311613011 103123 2318,F,239,110,1501.65,6,[1501.65],HCES202265365,1,03,0311613011 103123 2318,9009,471.428571,707920.714286


In [50]:
temp_dropped = temp1.drop(['Questionnaire No.','Item Code','Multiplier_x','count','Multiplier_y','part2','part3'],axis=1)
temp_dropped.head(5)

,Common-ID,Value (in Rs),Sector,State,total_mambers,Monthly Value,total_Value
12877,HCES2022653651030311613011 103123 2301,50,1,03,6006,214.285714,321782.142857
12887,HCES2022653651030311613011 103123 2302,60,1,03,4504,257.142857,386138.571429
12914,HCES2022653651030311613011 103123 2306,60,1,03,3003,257.142857,386138.571429
12989,HCES2022653651030311613011 103123 2316,50,1,03,7508,214.285714,321782.142857
13007,HCES2022653651030311613011 103123 2318,110,1,03,9009,471.428571,707920.714286


In [51]:
temp_dropped['per_person_per_month'] = temp_dropped['total_Value']/temp_dropped['total_mambers']
temp_dropped.head(5)

,Common-ID,Value (in Rs),Sector,State,total_mambers,Monthly Value,total_Value,per_person_per_month
12877,HCES2022653651030311613011 103123 2301,50,1,03,6006,214.285714,321782.142857,53.576780
12887,HCES2022653651030311613011 103123 2302,60,1,03,4504,257.142857,386138.571429,85.732365
12914,HCES2022653651030311613011 103123 2306,60,1,03,3003,257.142857,386138.571429,128.584273
12989,HCES2022653651030311613011 103123 2316,50,1,03,7508,214.285714,321782.142857,42.858570
13007,HCES2022653651030311613011 103123 2318,110,1,03,9009,471.428571,707920.714286,78.579278


In [52]:
temp_dropped.describe()

,Value (in Rs),total_mambers,Monthly Value,total_Value,per_person_per_month
count,239324.000000,239324.000000,239324.000000,2.393240e+05,239324.000000
mean,134.995161,4673.418203,578.550692,5.804946e+05,156.399517
std,121.726878,6260.723889,521.686621,8.859451e+05,159.536422
min,1.000000,4.000000,4.285714,2.905714e+02,1.648936
25%,53.000000,1633.000000,227.142857,1.492329e+05,60.000000
50%,100.000000,3956.000000,428.571429,3.555977e+05,107.219941
75%,177.000000,6508.000000,758.571429,7.384501e+05,196.446005
max,6900.000000,966006.000000,29571.428571,8.280051e+07,5895.879121


In [53]:
temp_dropped['Sector'].unique()

array(['1', '2'], dtype=object)

In [54]:
Urban_edu = temp_dropped[temp_dropped['Sector'].isin(['1'])]

In [55]:
Urban_edu.describe()

,Value (in Rs),total_mambers,Monthly Value,total_Value,per_person_per_month
count,139225.000000,139225.000000,139225.000000,1.392250e+05,139225.000000
mean,118.181605,5676.649517,506.492594,5.989708e+05,125.663223
std,105.917478,7438.403898,453.932048,8.915569e+05,122.113014
min,2.000000,4.000000,8.571429,2.905714e+02,1.648936
25%,50.000000,2846.000000,214.285714,1.834770e+05,51.428889
50%,90.000000,5185.000000,385.714286,3.975969e+05,87.861714
75%,150.000000,7506.000000,642.857143,7.738620e+05,157.311953
max,2000.000000,966006.000000,8571.428571,8.280051e+07,2489.138257


In [56]:
Rural_edu = temp_dropped[temp_dropped['Sector'].isin(['2'])]
Rural_edu.describe()

,Value (in Rs),total_mambers,Monthly Value,total_Value,per_person_per_month
count,100099.000000,100099.00000,100099.000000,1.000990e+05,100099.000000
mean,158.380683,3278.05082,678.774356,5.547966e+05,199.149799
std,137.415978,3662.07795,588.925619,8.774380e+05,192.277809
min,1.000000,8.00000,4.285714,4.026000e+02,1.904861
25%,65.000000,888.00000,278.571429,1.080522e+05,80.374865
50%,120.000000,2459.00000,514.285714,2.949673e+05,143.314286
75%,209.000000,4481.00000,895.714286,6.793200e+05,257.159355
max,6900.000000,137426.00000,29571.428571,3.459394e+07,5895.879121
